In [79]:
# always get fresh copies!
! cp ~cs186/sp16/hw2/part2/part2.db .
! chmod 700 part2.db
! cp ~cs186/sp16/hw2/part2/part2sampled.db .
! chmod 700 part2sampled.db
! cp ~cs186/sp16/hw2/part2/part2test.db .
! chmod 700 part2test.db

In [80]:
import subprocess, sqlite3, csv, os, sys

# jupyter weirdness; this makes sure that stdout is not buffered
def printNow(s):
    print s
    sys.stdout.flush()

    
def checkFile(f):
    if not (os.path.isfile(f)):
        raise IOError("DB File " + f + " is not here!")


currentDB1 = currentDB2 = "part2test.db"

# for grading dataset
# note that the two DB are different! currentDB1 is smaller in size
if ('CS186GRADER' in os.environ):
    currentDB1 = "part2sampled.db" 
    currentDB2 = "part2.db" 
    
checkFile(currentDB1)
dbQ1 = sqlite3.connect(currentDB1) 
dbQ1.text_factory = str

checkFile(currentDB2)
dbQ2 = sqlite3.connect(currentDB2)
dbQ2.text_factory = str

In [81]:
# Q1
# Degrees of separation
# Expected output: a paths table (as defined), note that we will be checking the original
#                  paths, and not just the average value

def initDegrees():
    dbQ1.executescript("""
        -- create paths table
        DROP TABLE IF EXISTS paths;
        DROP TABLE IF EXISTS pathsLastUpdated;
        DROP INDEX IF EXISTS paths_src_idx;
        DROP INDEX IF EXISTS paths_dst_idx;

        CREATE TABLE paths (
            src varchar(200),
            dst varchar(200),
            length integer,
            PRIMARY KEY (src, dst)
        );
        
        CREATE TABLE pathsLastUpdated (
            src varchar(200),
            dst varchar(200),
            length integer
        );

        CREATE INDEX paths_src_idx ON paths(src);
        CREATE INDEX paths_dst_idx ON paths(dst);
    """);
    dbQ1.commit()
    return


def populatePaths():
    dbQ1.executescript("""
      -- initialize paths with the first level connections
      INSERT INTO paths SELECT
        src,
        dst,
        1
      FROM links;      
      
      -- initialized for your convenience the "last udpated"
      INSERT INTO pathsLastUpdated SELECT
        src,
        dst,
        1
      FROM links;
    """);
    
    dbQ1.commit()
    return;


# Please implement the algorithm such that you don't repeat paths that were previously explored
# For example, for graph a->b->c->d, we want to see (a,b), (b,c), (c,d) then (a,c), (b,d), then (a,d)
# This makes a rather significant performance difference. To this end we have set up the 
#   pathsLastUpdated for you to store intermediate results
def getNewPaths():
    dbQ1.executescript("""
        -- this is an intermediate table for you to store temp data
        DROP TABLE IF EXISTS pathsNew;  
        CREATE TABLE pathsNew (
            src varchar(200),
            dst varchar(200),
            length integer
        );    
        
        INSERT INTO pathsNew        
            -- TODO (hint: use pathsLastUpdated)
            SELECT a.src, b.dst, length + 1 FROM pathsLastUpdated AS a 
                LEFT OUTER JOIN links AS b ON a.dst = b.src WHERE b.dst IS NOT NULL
                AND a.src != b.dst;
            
        -- ;
     """)
    dbQ1.commit()
    return

def updatePathsWithNewpaths():
    dbQ1.executescript("""
        DROP TABLE IF EXISTS pathsLastUpdated;
        DROP VIEW IF EXISTS temp2;
        
        CREATE VIEW temp2 AS
            SELECT DISTINCT * FROM pathsNew;
        
        -- add it to path
        INSERT INTO paths
        -- TODO (hint: use pathsNew)
            SELECT * FROM temp2 AS a 
                WHERE NOT EXISTS (SELECT * FROM paths AS b WHERE a.src = b.src AND a.dst = b.dst);
        -- ;
        
        ALTER TABLE pathsNew RENAME TO pathsLastUpdated;        
    """)
    dbQ1.commit()
    return


def getPathsCount():
    r = dbQ1.execute("SELECT COUNT(*) FROM paths;").fetchone()
    return r[0]


def getAveragePathsLength():
    r = dbQ1.execute("SELECT avg(length) FROM paths;").fetchone()
    return r[0]


def findDegreeDistribution():
    printNow("initializing degrees.  Calling sqlite...")
    initDegrees()
    printNow("Done, now populating paths.  Calling sqlite...")
    populatePaths()
    printNow("Done, now getting pathcount.  Calling sqlite...")
    oldCount = getPathsCount()
    
    i = 0
    while True:
        i += 1
        printNow("Beginning iteration #" + str(i) + ".  Calling sqlite ...")
        getNewPaths()
        printNow("Updating paths")
        updatePathsWithNewpaths()
        currentCount = getPathsCount()
        printNow("Done, current path total is: " + str(currentCount))
        if (currentCount == oldCount):
            break
        else:
            oldCount = currentCount

            
    print 'Converged! The average length of the shortest paths is: ', getAveragePathsLength()

findDegreeDistribution()

initializing degrees.  Calling sqlite...
Done, now populating paths.  Calling sqlite...
Done, now getting pathcount.  Calling sqlite...
Beginning iteration #1.  Calling sqlite ...
Updating paths
Done, current path total is: 3817
Beginning iteration #2.  Calling sqlite ...
Updating paths
Done, current path total is: 5542
Beginning iteration #3.  Calling sqlite ...
Updating paths
Done, current path total is: 7117
Beginning iteration #4.  Calling sqlite ...
Updating paths
Done, current path total is: 8549
Beginning iteration #5.  Calling sqlite ...
Updating paths
Done, current path total is: 9850
Beginning iteration #6.  Calling sqlite ...
Updating paths
Done, current path total is: 11025
Beginning iteration #7.  Calling sqlite ...
Updating paths
Done, current path total is: 12107
Beginning iteration #8.  Calling sqlite ...
Updating paths
Done, current path total is: 13099
Beginning iteration #9.  Calling sqlite ...
Updating paths
Done, current path total is: 13951
Beginning iteration #10

In [82]:
# Q2
# Expected output: nodes table populated with pagerank results 
# Note:
# - keeping track of the old PR value helps find convergence, though it's not required in the
#   assignment, it will help you see how close the values end up becoming.
def initPageRank():
    dbQ2.executescript("""
        DROP TABLE IF EXISTS nodes;
        CREATE TABLE nodes (
            node varchar(200) PRIMARY KEY,
            prevpr real,
            currentpr real,
            outdegree integer
        );
    """);
    dbQ2.commit()
    return


def populatePageRank():
    dbQ2.executescript("""
        INSERT INTO nodes
        SELECT
            DISTINCT links.src,
            1,
            1,
            COUNT(links.dst)
        FROM
            links
        GROUP BY
            links.src;
               
        -- corner case #1 nodes with no outgoing edges
        INSERT INTO
            nodes
        SELECT
            DISTINCT links.dst,
            1,
            1,
            0
        FROM
            links
        WHERE
            links.dst NOT IN (
                SELECT node FROM nodes
            );    
            
        -- corner cases #2 nodes with no incoming edges
        UPDATE
            nodes
        SET
            currentpr = 0.15
        WHERE
            nodes.node not in (
                SELECT DISTINCT dst
                FROM links
            );
    """);
    dbQ2.commit()

    return


def updatePageRank():

    dbQ2.executescript(""" 
        DROP VIEW IF EXISTS temp;
        DROP VIEW IF EXISTS nodesTemp;
        DROP VIEW IF EXISTS weight;
        DROP TABLE IF EXISTS nodes_holder;
        
        -- TODO (hint: you might want a helper table to store intermediate results)
        
        -- temporary table to hold nodes so that views won't get messed up
        CREATE TABLE nodes_holder (
            node varchar(200) PRIMARY KEY,
            prevpr real,
            currentpr real,
            outdegree integer
        );
        INSERT INTO nodes_holder 
            SELECT * FROM nodes;
        
        DELETE FROM nodes;
        
        -- temp is used to group up incoming nodes to their destinations
        CREATE VIEW temp AS
            SELECT * FROM nodes_holder AS a, links as b WHERE b.dst = a.node;
        
        -- pairs all nodes that are heading to same vertex and sums their normalized weight
        CREATE VIEW nodesTemp AS
            SELECT a.node , a.currentpr AS prevpr, SUM(b.currentpr/b.outdegree) as sum, a.outdegree 
            FROM temp AS a, nodes_holder AS b WHERE b.node = a.src GROUP BY a.node;
        
        -- creates total weight (add .15 + .85 * ...)
        CREATE VIEW weight AS
            SELECT node, prevpr, .15 + .85 * sum AS currentpr, outdegree FROM nodesTemp;
        
        -- add values from weight to nodes
        
        INSERT INTO nodes SELECT * FROM weight;
        INSERT INTO nodes SELECT * FROM nodes_holder WHERE node not in(SELECT DISTINCT dst FROM links);
            
    """);
    dbQ2.commit()

    return

def topPageRank():
    
    dbQ2.executescript("""
    
        DROP VIEW IF EXISTS topPR;
        CREATE VIEW topPR AS
        SELECT *
        FROM nodes
        ORDER BY currentpr DESC
        LIMIT 100;
        
    """);
    dbQ2.commit()

    return

def printTopPageRank():
    # we need to create an cursor object to fetch results to this program
    #    which we didn't need to before since it was just updating the DB
    r = dbQ2.execute("SELECT * FROM topPR;").fetchall()
    for i in r:
        print i
    

def runPageRank(N=10):
    printNow('Initializing page rank')
    initPageRank()
    printNow('Populating PageRank')
    populatePageRank()
    for i in xrange(N):
        printNow('Updating PageRank for iteration {}'.format(i))
        updatePageRank()
        
    printNow('Running top pagerank!')        
    topPageRank()
    printNow('Done! Here are your results')        
    printTopPageRank()
    
    return


runPageRank()

Initializing page rank
Populating PageRank
Updating PageRank for iteration 0
Updating PageRank for iteration 1
Updating PageRank for iteration 2
Updating PageRank for iteration 3
Updating PageRank for iteration 4
Updating PageRank for iteration 5
Updating PageRank for iteration 6
Updating PageRank for iteration 7
Updating PageRank for iteration 8
Updating PageRank for iteration 9
Running top pagerank!
Done! Here are your results
('United_States', 42.94909588388277, 43.086730852961665, 294)
('France', 28.824087322411664, 28.941022901988724, 85)
('Europe', 28.437573919393728, 28.545591377343516, 159)
('United_Kingdom', 27.985723871248915, 28.08984502464892, 168)
('English_language', 21.818058569049207, 21.902783517018044, 118)
('Germany', 21.638195302444558, 21.722961512744124, 169)
('World_War_II', 21.211186128106835, 21.29122959739671, 119)
('England', 20.09345568016141, 20.15586483349518, 172)
('Latin', 19.79615592340489, 19.86028286497774, 29)
('India', 18.106733503836278, 18.1825168

In [83]:
# you should have ran the two previous cells before running the test!
# make sure also that you have set the db to part2test.db!
! ./test.sh

Error: no such table: paths
ERROR q1! See your_output/q1.csv
Error: no such table: topPR
ERROR q2! See your_output/q2.csv
